In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# chaves da API
eduluz@gmail

103274251389094881747$f34de79813883385e129ca9cc225bfde235e75556bc5bdde7f5e4a56d37fc91f

eduluz@ufop

114275914826369140162$10dd060637cafc0588838170be9e21259d265962d7a4d3353d85112da3d3b0c3

alice@gmail

102967372526087716403$d8d4d59e5ee7cf9adee258414475cdb045ed3ded3315179fea2d13b172c4d791

In [2]:
import pandas as pd

# Substitua o 'caminho_do_arquivo.csv' pelo caminho real do seu arquivo CSV
arquivo_csv = '/content/drive/MyDrive/Colab Notebooks/2023/BRACIS_toxic/data/train_toldbr.csv'

# Ler o arquivo CSV
dados = pd.read_csv(arquivo_csv)

# Mostrar as primeiras linhas do DataFrame
print(dados.head())


                                                text  Toxic  \
0  Meu nivel de amizade com isis é ela ter meu in...    1.0   
1  rt @user @user o cara adultera dados, que fora...    1.0   
2  @user @user @user o cara só é simplesmente o m...    1.0   
3  eu to chorando vei vsf e eu nem staneio izone ...    1.0   
4  tem um do jack com a msm música e agr não sei ...    0.0   

                                          clean_text  
0  Meu nivel de amizade com isis é ela ter meu in...  
1  rt @user @user o cara adultera dados, que fora...  
2  @user @user @user o cara só é simplesmente o m...  
3  eu to chorando vei vsf e eu nem staneio izone ...  
4  tem um do jack com a msm música e agr não sei ...  


In [3]:
# Selecionar colunas específicas
dados_brutos = dados.iloc[:, 0]
rotulos = dados.iloc[:, 1]
dados_preprocessados = dados.iloc[:, 2]

# Mostrar as primeiras linhas de cada coluna
print("Dados brutos:")
print(dados_brutos.head())
print("\nRótulos:")
print(rotulos.head())
print("\nDados preprocessados:")
print(dados_preprocessados.head())


Dados brutos:
0    Meu nivel de amizade com isis é ela ter meu in...
1    rt @user @user o cara adultera dados, que fora...
2    @user @user @user o cara só é simplesmente o m...
3    eu to chorando vei vsf e eu nem staneio izone ...
4    tem um do jack com a msm música e agr não sei ...
Name: text, dtype: object

Rótulos:
0    1.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Toxic, dtype: float64

Dados preprocessados:
0    Meu nivel de amizade com isis é ela ter meu in...
1    rt @user @user o cara adultera dados, que fora...
2    @user @user @user o cara só é simplesmente o m...
3    eu to chorando vei vsf e eu nem staneio izone ...
4    tem um do jack com a msm música e agr não sei ...
Name: clean_text, dtype: object


Conta palavras no datasets de TESTE

In [4]:
import csv

def contar_palavras(arquivo_csv):
    numero_de_palavras = 0

    with open(arquivo_csv, 'r', encoding="utf-8") as arquivo:
        leitor = csv.reader(arquivo, delimiter=',')  # assumindo que a vírgula é o delimitador

        for linha in leitor:
            for campo in linha:
                numero_de_palavras += len(campo.split())

    return numero_de_palavras

arquivo_csv = '/content/drive/MyDrive/Colab Notebooks/2023/BRACIS_toxic/data/test_toldbr.csv'
total = contar_palavras(arquivo_csv)
print(f"O arquivo contém {total} palavras.")


O arquivo contém 134099 palavras.


Chama a APi do maritaca e faz a classificação

In [5]:
pip install maritalk

In [13]:
import pandas as pd
import time
import requests

# Configuração da API
KEY = '102967372526087716403$d8d4d59e5ee7cf9adee258414475cdb045ed3ded3315179fea2d13b172c4d791'
URL = "https://chat.maritaca.ai/api/chat/inference"
HEADERS = {
    "authorization": f"Key {KEY}"
}

# Leitura dos conjuntos de dados
arquivo_treinamento = '/content/drive/MyDrive/Colab Notebooks/2023/BRACIS_toxic/data/train_toldbr.csv'
arquivo_teste = '/content/drive/MyDrive/Colab Notebooks/2023/BRACIS_toxic/data/test_toldbr.csv'
dados_treinamento = pd.read_csv(arquivo_treinamento)
dados_teste = pd.read_csv(arquivo_teste)

# Seleção de exemplos few-shot
SEMENTE = 42
exemplos_toxicos = dados_treinamento[dados_treinamento['Toxic'] == 1.0].sample(50, random_state=SEMENTE)
exemplos_nao_toxicos = dados_treinamento[dados_treinamento['Toxic'] == 0.0].sample(50, random_state=SEMENTE)


# Criação do prompt com os exemplos
prompt_inicial = "Você está realizando análise de texto. O texto a seguir foi utilizado em um contexto de uma rede social. Responda apenas 'sim' ou 'não' caso o texto abaixo contenha discurso de ódio, ofensa, agressividade ou outras formas de toxicidade:\n"
for _, row in exemplos_toxicos.iterrows():
    prompt_inicial += f"Texto: {row['clean_text']} -> Resposta: sim\n"

for _, row in exemplos_nao_toxicos.iterrows():
    prompt_inicial += f"Texto: {row['clean_text']} -> Resposta: não\n"
prompt_inicial += "Agora classifique o texto a seguir respondendo com 'sim' ou 'não': "

# Função para classificar texto com few-shot
def classificar_texto(texto):
    prompt_completo = prompt_inicial + texto

    def contar_palavras(texto):
      return len(texto.split())

    numero_de_palavras = contar_palavras(prompt_inicial)
    print(f"O prompt inicial contém {numero_de_palavras} palavras.")

    numero_de_palavras = contar_palavras(prompt_completo)
    print(f"O prompt completo contém {numero_de_palavras} palavras.")

    data = {
        "messages": [
            {"role": "user", "content": prompt_completo}
        ],
        "do_sample": False,           # Determinístico
        "temperature": 0,             # Mais determinístico
        #"max_tokens": 8000,            # Máximo de tokens (ajustado para lidar com o prompt maior)
        "model": "maritalk",          # Modelo padrão
        "repetition_penalty": 1.2     # Penalidade de repetição
    }

    resposta = requests.post(URL, headers=HEADERS, json=data)

    if resposta.status_code == 429:
        print("Rate limited, tente novamente em breve.")
        return None

    elif resposta.status_code == 200:
        return resposta.json()['answer']

    else:
        print(f"Erro na requisição: {resposta.status_code} - {resposta.text}")
        return None

# Classificar cada tweet usando few-shot
resultados = []

for tweet in dados_teste['clean_text']:
    resultado = classificar_texto(tweet)
    print(tweet)
    print(resultado + '\n')
    if resultado:
        resultados.append(resultado)
    time.sleep(5)  # delay de 5 segundos entre as requisições

# Salvar resultados
dados_teste['Classificação_MariTalk'] = resultados
dados_teste.to_csv('/content/drive/MyDrive/Colab Notebooks/2023/BRACIS_toxic/data/resultados_classificacao_maritaca_ToldBR_prompt_2_fewshot.csv', index=False)
print("Classificação concluída e resultados salvos.")


O prompt inicial contém 2113 palavras.
O prompt completo contém 2121 palavras.
rt @user a maior ilusão coletiva do twitter
Não

O prompt inicial contém 2113 palavras.
O prompt completo contém 2121 palavras.
porra brabão sair pra comprar roupa nessa chuva
sim

O prompt inicial contém 2113 palavras.
O prompt completo contém 2116 palavras.
@user ridículo isso
Não

O prompt inicial contém 2113 palavras.
O prompt completo contém 2125 palavras.


KeyboardInterrupt: ignored